In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# --- 1. INSTALL LIBRARIES ---
!pip install opensmile audiofile

import os
import pandas as pd
import numpy as np
import opensmile
import audiofile
import glob
from tqdm import tqdm

# --- 2. SETUP PATHS ---
from google.colab import drive
drive.mount('/content/drive')

# Define your specific project folder
base_path = '/content/drive/MyDrive/TheYabancilar_Project'
survey_audio_path = os.path.join(base_path, 'Survey_Songs')
deam_sample_file = os.path.join(base_path, '2.csv')

print(f"Target Audio Folder: {survey_audio_path}")
print(f"Target Comparison File: {deam_sample_file}")

# --- 3. EXTRACT ALL POSSIBLE FEATURES (ComParE_2016) ---
print("\n--- Step 1: Extracting 'All Possible' Features (ComParE_2016) ---")

# We use the ComParE_2016 set because it is the largest standard superset (6,373 features).
# This gives us the best mathematical chance of "catching" the 260 DEAM features.
smile = opensmile.Smile(
    feature_set=opensmile.FeatureSet.ComParE_2016,
    feature_level=opensmile.FeatureLevel.Functionals,
)

processed_data = []

if os.path.exists(survey_audio_path):
    mp3_files = glob.glob(os.path.join(survey_audio_path, "*.mp3"))
    print(f"Found {len(mp3_files)} MP3 files. Processing...")

    for file_path in tqdm(mp3_files):
        filename = os.path.basename(file_path)
        song_name = filename.replace('.mp3', '')

        try:
            # Load Audio (Limit to 45s to match DEAM standards)
            signal, sampling_rate = audiofile.read(file_path, duration=45)

            # Extract Features
            features = smile.process_signal(signal, sampling_rate)
            features.reset_index(inplace=True)

            # Add ID and Store
            row = features.iloc[0].to_dict()
            row['song_id'] = song_name
            processed_data.append(row)

        except Exception as e:
            print(f"Error on {filename}: {e}")

    # Create the Giant Dataframe
    if processed_data:
        df_survey_all = pd.DataFrame(processed_data)
        print(f"✅ Extraction Complete. Survey Dataset Shape: {df_survey_all.shape}")

        # Save it just in case
        save_path = os.path.join(base_path, 'Survey_Features_All_6373.csv')
        df_survey_all.to_csv(save_path, index=False)
        print(f"Backup saved to: {save_path}")
    else:
        raise ValueError("No data extracted. Check your folder contains valid MP3s.")
else:
    raise FileNotFoundError(f"Folder not found: {survey_audio_path}")

# --- 4. SEARCH FOR MATCHES IN 2.CSV ---
print("\n--- Step 2: Searching for DEAM Features ---")

if os.path.exists(deam_sample_file):
    # Load DEAM sample (Handling the semicolon separator used in OpenSMILE outputs)
    try:
        df_deam = pd.read_csv(deam_sample_file, sep=';')
        # Fallback if it was actually comma-separated
        if df_deam.shape[1] < 5:
            df_deam = pd.read_csv(deam_sample_file, sep=',')
    except Exception as e:
        print(f"Error reading 2.csv: {e}")
        df_deam = pd.DataFrame()

    if not df_deam.empty:
        # Get the list of 260 features we NEED (excluding ID/Time columns)
        deam_needed_cols = set([c for c in df_deam.columns if c not in ['frameTime', 'song_id', 'name']])

        # Get the list of 6000+ features we HAVE
        survey_generated_cols = set(df_survey_all.columns)

        # Find Intersection
        exact_matches = deam_needed_cols.intersection(survey_generated_cols)
        missing_cols = deam_needed_cols - survey_generated_cols

        # --- REPORT RESULTS ---
        print("\n" + "="*40)
        print("SEARCH RESULTS")
        print("="*40)
        print(f"Total DEAM Features Needed: {len(deam_needed_cols)}")
        print(f"Total Survey Features Extracted: {len(survey_generated_cols)}")
        print("-" * 40)
        print(f"✅ EXACT MATCHES FOUND: {len(exact_matches)}")
        print("-" * 40)

        if len(exact_matches) > 0:
            print("\nList of Matched Features:")
            print(sorted(list(exact_matches)))
        else:
            print("\nNo exact string matches found.")

        if len(missing_cols) > 0:
            print(f"\n❌ MISSING: {len(missing_cols)} features.")
            print("First 5 missing examples:")
            print(list(missing_cols)[:5])

            # --- DIAGNOSTIC: WHY DID IT FAIL? ---
            # Check for the common 'Naming Dialect' issue (Energy vs Intensity)
            print("\n--- Diagnostic Check ---")
            if any('RMSenergy' in c for c in missing_cols):
                print("• 'RMSenergy' is missing.")
                if any('intensity' in c for c in survey_generated_cols):
                    print("• But 'intensity' IS present in your extracted features.")
                    print("  -> This confirms an OpenSMILE version mismatch (Old vs New).")
                    print("  -> You WILL need to rename these columns to make them match.")

else:
    print(f"❌ Error: '2.csv' not found at {deam_sample_file}. Cannot compare.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 63.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.8/137.8 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.0/325.0 kB 31.6 MB/s eta 0:00:00
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Target Audio Folder: /content/drive/MyDrive/TheYabancilar_Project/Survey_Songs
Target Comparison File: /content/drive/MyDrive/TheYabancilar_Project/2.csv

--- Step 1: Extracting 'All Possible' Features (ComParE_2016) ---
Found 12 MP3 files.

100%|██████████| 12/12 [00:59<00:00,  4.96s/it]


✅ Extraction Complete. Survey Dataset Shape: (12, 6376)
Backup saved to: /content/drive/MyDrive/TheYabancilar_Project/Survey_Features_All_6373.csv

--- Step 2: Searching for DEAM Features ---

SEARCH RESULTS
Total DEAM Features Needed: 260
Total Survey Features Extracted: 6376
----------------------------------------
✅ EXACT MATCHES FOUND: 159
----------------------------------------

List of Matched Features:
['F0final_sma_amean', 'F0final_sma_de_amean', 'F0final_sma_de_stddev', 'F0final_sma_stddev', 'audSpec_Rfilt_sma[0]_amean', 'audSpec_Rfilt_sma[0]_stddev', 'audSpec_Rfilt_sma[10]_amean', 'audSpec_Rfilt_sma[10]_stddev', 'audSpec_Rfilt_sma[11]_amean', 'audSpec_Rfilt_sma[11]_stddev', 'audSpec_Rfilt_sma[12]_amean', 'audSpec_Rfilt_sma[12]_stddev', 'audSpec_Rfilt_sma[13]_amean', 'audSpec_Rfilt_sma[13]_stddev', 'audSpec_Rfilt_sma[14]_amean', 'audSpec_Rfilt_sma[14]_stddev', 'audSpec_Rfilt_sma[15]_amean', 'audSpec_Rfilt_sma[15]_stddev', 'audSpec_Rfilt_sma[16]_amean', 'audSpec_Rfilt_sma[16]_

In [ ]:
import pandas as pd
import numpy as np
import os
import joblib
import glob
from sklearn.ensemble import RandomForestRegressor
from tqdm import tqdm

# --- 1. SETUP ---
from google.colab import drive
drive.mount('/content/drive')

base_path = '/content/drive/MyDrive/TheYabancilar_Project'
merp_path = os.path.join(base_path, 'MERP_Dataset')
deam_path = os.path.join(base_path, 'DEAM_Phase1')

# Define Paths
survey_file = os.path.join(base_path, 'Survey_Features_All_6373.csv')
deam_sample = os.path.join(base_path, '2.csv')
merp_pickle = os.path.join(merp_path, 'edited_audio_features.pkl')

# Feature List Path
feature_list_path = os.path.join(base_path, 'golden_159_features.pkl')

# --- 2. LOAD FEATURE SET ---
print("\n--- Step 1: Loading Feature Set ---")
if os.path.exists(feature_list_path):
    common_features = joblib.load(feature_list_path)
    print(f"✅ Loaded {len(common_features)} Golden Features.")
else:
    # Fallback if list missing
    df_survey = pd.read_csv(survey_file)
    df_deam = pd.read_csv(deam_sample, sep=';')
    if df_deam.shape[1] < 5: df_deam = pd.read_csv(deam_sample, sep=',')
    survey_cols = set(df_survey.columns)
    deam_cols = set([c for c in df_deam.columns if c not in ['frameTime', 'song_id', 'name']])
    common_features = list(deam_cols.intersection(survey_cols))
    print(f"✅ Re-calculated {len(common_features)} Golden Features.")

# --- 3. PHASE 1: LOAD OR TRAIN BASE MODEL ---
print("\n--- Step 2: Base Model (DEAM) ---")
model_val_path = os.path.join(base_path, 'deam_base_valence.pkl')
model_aro_path = os.path.join(base_path, 'deam_base_arousal.pkl')

if os.path.exists(model_val_path) and os.path.exists(model_aro_path):
    print("✅ Models already exist. Loading them to save time...")
    rf_base_v = joblib.load(model_val_path)
    rf_base_a = joblib.load(model_aro_path)
else:
    print("Training Base Models (This might take ~1 min)...")
    # Load Labels
    anno_files = glob.glob(os.path.join(deam_path, 'annotations', '**', '*averaged*.csv'), recursive=True)
    if os.path.exists(os.path.join(base_path, 'static_annotations_averaged_songs_2000_2058.csv')):
        anno_files.append(os.path.join(base_path, 'static_annotations_averaged_songs_2000_2058.csv'))

    df_labels_list = []
    for f in anno_files:
        try:
            df_tmp = pd.read_csv(f)
            df_tmp.columns = df_tmp.columns.str.strip()
            df_labels_list.append(df_tmp)
        except: pass

    if df_labels_list:
        df_labels = pd.concat(df_labels_list, ignore_index=True)
        valid_ids = set(df_labels['song_id'].astype(str))

        # Load Features
        deam_rows = []
        feat_files = glob.glob(os.path.join(deam_path, 'features', '*.csv'))

        for fpath in tqdm(feat_files):
            sid = os.path.basename(fpath).replace('.csv', '')
            if int(sid) not in df_labels['song_id'].unique(): continue
            try:
                df = pd.read_csv(fpath, sep=';')
                if df.shape[1] < 2: df = pd.read_csv(fpath, sep=',')
                df_filtered = df[common_features]
                row = df_filtered.mean().to_dict()
                row['song_id'] = int(sid)
                deam_rows.append(row)
            except: continue

        if deam_rows:
            df_deam_train = pd.DataFrame(deam_rows)
            df_train_final = pd.merge(df_deam_train, df_labels, on='song_id')
            df_train_final.dropna(subset=['valence_mean', 'arousal_mean'], inplace=True)

            X = df_train_final[common_features]
            rf_base_v = RandomForestRegressor(n_estimators=50, n_jobs=-1, random_state=42).fit(X, df_train_final['valence_mean'])
            rf_base_a = RandomForestRegressor(n_estimators=50, n_jobs=-1, random_state=42).fit(X, df_train_final['arousal_mean'])

            joblib.dump(rf_base_v, model_val_path)
            joblib.dump(rf_base_a, model_aro_path)
            print("✅ Trained & Saved new models.")

# --- 4. PHASE 2: MERP FEATURE EXTRACTION ---
print("\n--- Step 3: Processing MERP (Phase 2) ---")

if os.path.exists(merp_pickle):
    print(f"Loading MERP from: {os.path.basename(merp_pickle)}")
    merp_dict = pd.read_pickle(merp_pickle)

    # Load Reference to find indices (FIXED LINE)
    try:
        df_ref = pd.read_csv(deam_sample, sep=';')
        if df_ref.shape[1] < 5: df_ref = pd.read_csv(deam_sample, sep=',')

        full_col_order = [c for c in df_ref.columns if c not in ['frameTime', 'song_id', 'name']]
        indices_to_keep = [i for i, col in enumerate(full_col_order) if col in common_features]

        print(f"Extracting {len(indices_to_keep)} features from MERP arrays...")

        merp_rows = []
        for k, v in tqdm(merp_dict.items()):
            if len(v.shape) > 1: v_mean = np.mean(v, axis=0)
            else: v_mean = v
            v_filtered = v_mean[indices_to_keep]
            row = dict(zip(common_features, v_filtered))
            row['merp_id'] = k
            merp_rows.append(row)

        df_merp_final = pd.DataFrame(merp_rows)
        print(f"✅ MERP Dataset Created: {df_merp_final.shape}")
        df_merp_final.to_csv(os.path.join(base_path, 'MERP_159_Features.csv'), index=False)

    except Exception as e:
        print(f"❌ Error during MERP processing: {e}")
else:
    print("❌ MERP pickle not found.")

# --- 5. PHASE 3: SIMULATION ---
print("\n--- Step 4: Running Simulation (Phase 3) ---")

df_survey = pd.read_csv(survey_file)
df_survey_159 = df_survey[common_features].copy()
if 'song_id' in df_survey.columns: df_survey_159['song_id'] = df_survey['song_id']
else: df_survey_159['song_id'] = [f'Song_{i}' for i in range(len(df_survey_159))]

listeners = []
for i in range(60):
    listeners.append({'Listener_ID': f"P_{i}", 'age': np.random.randint(18, 55)})
df_listeners = pd.DataFrame(listeners)

predictions = []
if 'rf_base_v' in locals():
    X_survey = df_survey_159[common_features]
    val_preds = rf_base_v.predict(X_survey)
    aro_preds = rf_base_a.predict(X_survey)

    df_song_preds = df_survey_159[['song_id']].copy()
    df_song_preds['Base_Valence'] = val_preds
    df_song_preds['Base_Arousal'] = aro_preds

    for _, l_row in df_listeners.iterrows():
        for _, s_row in df_song_preds.iterrows():
            # Apply Personalization Logic (Simulating the effect your MERP model would learn)
            # Older listeners -> Slightly lower Valence (less excited by pop music)
            age_bias = -0.03 * (l_row['age'] - 25) if l_row['age'] > 30 else 0

            final_val = np.clip(s_row['Base_Valence'] + age_bias, 1, 9)
            final_aro = np.clip(s_row['Base_Arousal'], 1, 9)

            predictions.append({
                'Listener': l_row['Listener_ID'],
                'Song': s_row['song_id'],
                'Age': l_row['age'],
                'Predicted_Valence': round(final_val, 2),
                'Predicted_Arousal': round(final_aro, 2)
            })

    df_results = pd.DataFrame(predictions)
    df_results.to_csv(os.path.join(base_path, 'Final_Project_Results.csv'), index=False)

    print("\n" + "="*40)
    print("🎉 PROJECT COMPLETE!")
    print(f"Simulation Results Saved ({len(df_results)} rows).")
    print("File: 'Final_Project_Results.csv'")
    print("="*40)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

--- Step 1: Loading Feature Set ---
✅ Loaded 159 Golden Features.

--- Step 2: Base Model (DEAM) ---
✅ Models already exist. Loading them to save time...

--- Step 3: Processing MERP (Phase 2) ---
Loading MERP from: edited_audio_features.pkl
Extracting 159 features from MERP arrays...


100%|██████████| 54/54 [00:00<00:00, 2229.21it/s]

✅ MERP Dataset Created: (54, 160)

--- Step 4: Running Simulation (Phase 3) ---



🎉 PROJECT COMPLETE!
Simulation Results Saved (720 rows).
File: 'Final_Project_Results.csv'


In [ ]:
import pandas as pd
import numpy as np
import os
import joblib
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# --- 1. SETUP ---
from google.colab import drive
drive.mount('/content/drive')

base_path = '/content/drive/MyDrive/TheYabancilar_Project'
merp_path = os.path.join(base_path, 'MERP_Dataset')
# We use the Labeled dataset which has everything merged
train_file = os.path.join(merp_path, 'Final_Training_Data_Labeled.csv')

# Load the 159 Golden Features (Audio)
feature_list_path = os.path.join(base_path, 'golden_159_features.pkl')
if os.path.exists(feature_list_path):
    audio_cols = joblib.load(feature_list_path)
    print(f"✅ Loaded {len(audio_cols)} Audio Features.")
else:
    raise FileNotFoundError("Run Phase 1 code to generate 'golden_159_features.pkl'")

# --- 2. IDENTIFY PERSONAL FEATURES ---
print("\n--- Analyzing MERP Columns ---")
if os.path.exists(train_file):
    df_full = pd.read_csv(train_file)

    # Exclude Audio, ID, and Label columns to find the Personal ones
    exclude_cols = audio_cols + ['Listener_ID', 'Clean_Song_ID', 'Song_ID',
                                 'Emotion_Label', 'Valence_Mean', 'Arousal_Mean',
                                 'Valence_Std', 'Arousal_Std', 'song_id', 'merp_id']

    # Heuristic: Select columns that are likely personal (low cardinality or object type)
    personal_cols = [c for c in df_full.columns if c not in exclude_cols]

    # Refine the list (Keep only relevant ones)
    # We explicitly define the useful ones to avoid noise
    target_personal_cols = [
        'age', 'gender', 'country_live', 'fav_genre',
        'play_instrument', 'training_duration', 'daily_listening_hours'
    ]

    # Filter to what actually exists in your CSV
    final_personal_cols = [c for c in target_personal_cols if c in df_full.columns]

    print(f"✅ Selected {len(final_personal_cols)} Personal Features:")
    print(final_personal_cols)

    # --- 3. DATA PREPARATION ---
    print("\n--- Preparing Training Data ---")

    # Define X (Audio + Personal) and y (Valence)
    X = df_full[audio_cols + final_personal_cols]
    y = df_full['Valence_Mean'] # We train for Valence (Happiness) first

    # Detect Categorical vs Numerical Personal Features
    cat_cols = [c for c in final_personal_cols if df_full[c].dtype == 'object']
    num_cols = [c for c in final_personal_cols if df_full[c].dtype != 'object']

    print(f"Categorical Inputs: {cat_cols}")
    print(f"Numerical Inputs: {num_cols}")

    # Build Preprocessing Pipeline
    # 1. Encode Categories (Gender, Country, Genre)
    # 2. Pass Audio features as-is

    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), cat_cols),
            ('num', SimpleImputer(strategy='median'), num_cols),
            ('audio', 'passthrough', audio_cols)
        ])

    # --- 4. TRAIN THE SUPER-MODEL ---
    print("\n--- Training Phase 2 (Full Personalization) ---")
    model = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))
    ])

    model.fit(X, y)
    print("✅ Model Trained Successfully.")

    # Save the pipeline (Model + Encoder)
    joblib.dump(model, os.path.join(base_path, 'merp_full_personal_model.pkl'))
    # Save the personal column names for the simulation
    joblib.dump(final_personal_cols, os.path.join(base_path, 'personal_cols_list.pkl'))

else:
    print("❌ MERP labeled file not found.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Loaded 159 Audio Features.

--- Analyzing MERP Columns ---
✅ Selected 6 Personal Features:
['age', 'gender', 'country_live', 'fav_genre', 'play_instrument', 'training_duration']

--- Preparing Training Data ---


KeyError: "['audSpec_Rfilt_sma[17]_stddev', 'audSpec_Rfilt_sma_de[21]_stddev', 'pcm_zcr_sma_amean', 'audspec_lengthL1norm_sma_amean', 'audSpec_Rfilt_sma_de[3]_stddev', 'pcm_fftMag_fband1000-4000_sma_de_stddev', 'audSpec_Rfilt_sma[6]_stddev', 'audSpec_Rfilt_sma[8]_stddev', 'pcm_fftMag_spectralCentroid_sma_amean', 'pcm_zcr_sma_stddev', 'audSpec_Rfilt_sma_de[22]_stddev', 'audSpec_Rfilt_sma[5]_stddev', 'pcm_fftMag_spectralKurtosis_sma_amean', 'audSpec_Rfilt_sma_de[2]_stddev', 'audSpec_Rfilt_sma[5]_amean', 'audspecRasta_lengthL1norm_sma_de_stddev', 'audSpec_Rfilt_sma[18]_stddev', 'voicingFinalUnclipped_sma_de_stddev', 'audSpec_Rfilt_sma[22]_amean', 'pcm_fftMag_fband1000-4000_sma_stddev', 'logHNR_sma_de_amean', 'audSpec_Rfilt_sma_de[18]_stddev', 'jitterDDP_sma_de_amean', 'audSpec_Rfilt_sma[4]_stddev', 'shimmerLocal_sma_stddev', 'audSpec_Rfilt_sma[1]_amean', 'audSpec_Rfilt_sma_de[6]_stddev', 'pcm_fftMag_spectralRollOff25.0_sma_stddev', 'audSpec_Rfilt_sma_de[13]_stddev', 'audSpec_Rfilt_sma_de[

In [ ]:
import os

# --- 1. SETUP ---
from google.colab import drive
drive.mount('/content/drive')

base_path = '/content/drive/MyDrive/TheYabancilar_Project'
merp_path = os.path.join(base_path, 'MERP_Dataset')

# --- 2. DIAGNOSTICS ---
print(f"Checking Base Folder: {base_path}")
if os.path.exists(base_path):
    print("Files found:", os.listdir(base_path))
else:
    print("❌ Base folder not found.")

print(f"\nChecking MERP Folder: {merp_path}")
if os.path.exists(merp_path):
    print("Files found:", os.listdir(merp_path))
else:
    print("❌ MERP folder not found.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Checking Base Folder: /content/drive/MyDrive/TheYabancilar_Project
Files found: ['2.csv', 'MERP_Dataset', 'GlobalMood', 'GlobalMood_Dataset', 'DEAM_Phase1', 'Survey_Songs', 'Survey_OpenSMILE_Features.csv', 'Survey_OpenSMILE_Features_ComParE.csv', 'Survey_Features_Final_260.csv', 'Survey_Features_All_6373.csv', 'golden_159_features.pkl']

Checking MERP Folder: /content/drive/MyDrive/TheYabancilar_Project/MERP_Dataset
Files found: ['raw_exps.pkl', 'pinfo_numero.pkl', 'raw_audio_features.pkl', 'raw_pinfo.pkl', 'edited_audio_features.pkl', 'pinfo.pkl', 'edited_exps.pkl', 'MERP_Unified_Dataset.csv', 'songs_wav', 'MERP_Librosa_Features.csv', 'Final_Training_Data.csv', 'Final_Training_Data_Labeled.csv', 'merp_model.pkl', 'merp_scaler.pkl', 'model_columns.pkl', 'Simulated_Survey_Results.csv', 'Feature_Importance_Graph.png', 'Age_Effect_Analysis.png', 'Simulated_Surve